In [259]:
import numpy as np
from numpy.linalg import multi_dot

import pandas as pd

In [260]:
# Building random data matrix R and obtain coefficient matrix C.

M = 10
N = 20

# Obserbed and unobserbed weights.
wk = 0.7
w0 = 0.5

R = np.random.choice([0, 1], size = (M, N), p = [3./5, 2./5])

c = [ sum(R[:, i]) for i in range(0, np.shape(R)[1]) ]
C = R * c * wk + w0

In [261]:
R

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1],
       [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0]])

In [262]:
# Building random X and Y matrices of proper dimension.

K = 5

X = np.random.rand(M, K)
Y = np.random.rand(N, K)

In [263]:
# Define functions to compute approximated ratings and error.

def approximation(X, Y):
    return np.dot(X, Y.T)

def error(approxRatings, ratings, w0):
    obs_idx = np.where(ratings > 0)
    nobs_idx = np.where(ratings == 0)
    obs_error = sum( (ratings[obs_idx] - approxRatings[obs_idx]) ** 2 )
    nobs_error = sum( (ratings[nobs_idx] - approxRatings[nobs_idx]) ** 2 )
    return obs_error + w0 * nobs_error

In [264]:
# Testing functions.

approx_ratings = approximation(X, Y)
w0 = 0.5
error(approx_ratings, R, w0)

163.6857301606067

In [265]:
def singlePassWALS(R, X, Y, C, reg_lambda):
    M = np.shape(X)[0]
    K = np.shape(X)[1]
    N = np.shape(Y)[0]

    for u in range(1, M):
        Cu = np.diag(C[u, :])
        # A X_u = b
        A = multi_dot([Y.T, Cu, Y]) + reg_lambda * np.eye(K)
        b = multi_dot([Y.T, Cu, R[u, :]])
        X_u = np.linalg.solve(A, b)
        
        X[u,] = X_u
        
    for i in range(1, N):
        Ci = np.diag(C[:,i])
        # A Y_i = b
        A = multi_dot([X.T, Ci, X]) + reg_lambda * np.eye(K)
        b = multi_dot([X.T, Ci, R[:, i]])
        Y_i = np.linalg.solve(A, b)
        
        Y[i,] = Y_i    
    

In [266]:
def WALS(R, X, Y, C, reg_lambda, n_iter):
    for j in range(1, n_iter):
        singlePassWALS(R, X, Y, C, reg_lambda)
        approx_ratings = approximation(X, Y)
        w0 = 0.5
        print("Error: " + str(error(approx_ratings, R, w0)))
        

In [267]:
# Compute WALS.

reg_lambda = 0.2
n_iter = 10


WALS(R, X, Y, C, reg_lambda, n_iter)

final_result = approximation(X, Y)

Error: 22.62233629571566
Error: 15.025697036674076
Error: 13.341552737737258
Error: 12.573144941035142
Error: 12.117414113017443
Error: 11.81167392398889
Error: 11.6059195420799
Error: 11.476406035365862
Error: 11.400516278725952


In [273]:
np.matrix.round(final_result, 1)

array([[ 1.4,  0.9,  0.2,  0.2,  0.6, -0. ,  0.9,  0.2,  0.4,  0.9,  0. ,
         1. ,  1.1,  0.1,  1.1, -0.1,  1. ,  0.9,  0.5,  0.9],
       [-0.2, -0.2,  0.9,  0.9,  0.9,  0.9, -0. ,  0.9,  0.3,  1. ,  0.1,
        -0. ,  0.1,  1.1,  1. ,  0. ,  0.9,  0.1,  0.4,  1. ],
       [ 0.9,  0.1,  0.1,  0.2,  0.9,  0.1,  1. ,  0.2,  1. ,  0.9,  0. ,
         0.2,  0.9,  0.7,  1. ,  0.6,  0.1,  0.2,  1. ,  0.8],
       [ 0.2,  0.2,  0.4,  0.9,  0.1,  0.9,  0.8,  0.9,  1.1,  0.2,  0.9,
         1. ,  0.1, -0.2,  0. ,  0.4,  0.7,  0.6,  0.5, -0.4],
       [-0.1,  0.4,  0.3,  0.3,  1.1,  0.2,  0. ,  0.3, -0.1,  0.5, -0.3,
         0.1,  0.2,  0.9,  1. ,  0.1,  0.9,  0.3, -0.2,  1. ],
       [ 0.8, -0. , -0. ,  0.1,  0.9, -0.1,  1. ,  0.1,  1. ,  0.8, -0.1,
        -0. ,  0.9,  0.8,  1. ,  0.8, -0.2,  0.1,  1. ,  0.9],
       [ 0.1,  0.9, -0.2,  0.3,  1. ,  0.2,  1.1,  0.4,  0.9, -0.3,  0.3,
         1. ,  0.3,  0.1,  0.5,  0.9,  0.9,  0.9, -0.1, -0. ],
       [ 0.9, -0. ,  0.8,  1.1, -0.2,  0.